# Limpieza de datos de produccion mensual agricola de la pagina del SIAP

Una vez que tengamos los datos en archivos CSV, realizaremos una pequeña exploración para llevar a cabo una limpieza sencilla. Esto implicará identificar datos faltantes, buscar instancias duplicadas y asegurarnos de que el tipo de dato de las columnas sea coherente.

### Librerías

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def custom_to_float(value):
    '''
    Función que transformá cadenas de caracteres que son 
    números separados por comas en flotantes.
    '''
    if isinstance(value, str):
        return float(value.replace(',', ''))
    elif isinstance(value, (int, float)):
        return float(value)
    else:
        return None

In [36]:
def update_df_SIAP(file_SIAP, df_new, replace = False):
    '''
    Función para actualizar nuestra base de datos del SIAP,
    si no existe se crea con el data frame proporcionado.
    Si quieres reemplazar el archivo haz replace = True.
    '''

    if replace:
        os.remove(file_SIAP)

    if not os.path.exists(file_SIAP):
        df_new.to_csv(file_SIAP, index=False, encoding='utf-8')
    else:
        df_new.to_csv(file_SIAP, mode='a', header=False, index=False)

Empezamos leyendo cada uno de los archivos y concatenándolos en un data frame

In [37]:
request_dict = {
  'anio': ['2020','2021','2022','2023'],
  'mes': ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto',
          'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'],
  'ciclo': ['Ciclicos - Perennes'],
  'modalidad': ['Riego + Temporal'],
  'cultivo': ['Tomate rojo (jitomate)', 'Chile verde', 'Limón', 'Pepino', 'Plátano', 'Mango', 'Brócoli', 
              'Cebolla', 'Sandía', 'Papaya', 'Calabacita', 'Lechuga', 'Nopalitos', 'Nuez', 'Fresa', 
              'Toronja (pomelo)', 'Piña', 'Berenjena', 'Uva', 'Naranja', 'Papa', 'Melón', 'Manzana', 
              'Pera', 'Durazno', 'Espárrago', 'Zarzamora', 'Coliflor', 'Guayaba', 'Tomate verde',
              'Frijol' , 'Garbanzo grano', 'Frambuesa'
              ]
}

directory = '../descarga/datosSIAP'
df_temp = pd.DataFrame()

for year in request_dict['anio']:

  for month in request_dict['mes']:

    for cycle in request_dict['ciclo']:

      for modality in request_dict['modalidad']:
        
        for crop in request_dict['cultivo']:  
          
          try:

            df = pd.read_csv(os.path.join(directory,(f"{year.replace(' ', '')}_{month.replace(' ', '')}" +
            f"_{cycle.replace(' ', '')}_{modality.replace(' ', '')}" +
            f"_{crop.replace(' ', '')}.csv")))
          
            if df_temp.empty:
              df_temp = df
            else:
              df_temp = pd.concat([df_temp, df], axis=0)

          except Exception as e:
            print(f"Failed: {str(e)}")
            continue
            
df_temp = df_temp.reset_index(drop=True)

Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2020_Enero_Ciclicos-Perennes_Riego+Temporal_Pepino.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2020_Enero_Ciclicos-Perennes_Riego+Temporal_Calabacita.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2020_Febrero_Ciclicos-Perennes_Riego+Temporal_Pepino.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2020_Febrero_Ciclicos-Perennes_Riego+Temporal_Calabacita.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2020_Marzo_Ciclicos-Perennes_Riego+Temporal_Pepino.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2020_Marzo_Ciclicos-Perennes_Riego+Temporal_Calabacita.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2020_Abril_Ciclicos-Perennes_Riego+Temporal_Pepino.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2020_Abril_Ciclicos-Perennes_Riego+Tempo

In [38]:
df_temp.head()

,Año,Mes,Ciclo,Modalidad,Cultivo,Entidad,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)
0,2020,Enero,Ciclicos - Perennes,Riego + Temporal,Tomate rojo (jitomate),Aguascalientes,Aguascalientes,Calvillo,16.00,6.00,0.0,90.00,15.00
1,2020,Enero,Ciclicos - Perennes,Riego + Temporal,Tomate rojo (jitomate),Baja California,Ensenada,Ensenada,19.50,0.00,0.0,0.00,0.00
2,2020,Enero,Ciclicos - Perennes,Riego + Temporal,Tomate rojo (jitomate),Baja California Sur,Mulegé,Mulegé,80.00,0.00,0.0,0.00,0.00
3,2020,Enero,Ciclicos - Perennes,Riego + Temporal,Tomate rojo (jitomate),Baja California Sur,Comondú,Comondú,127.00,0.00,0.0,0.00,0.00
4,2020,Enero,Ciclicos - Perennes,Riego + Temporal,Tomate rojo (jitomate),Baja California Sur,La Paz,La Paz,611.00,106.00,0.0,"4,429.76",41.79


Empezaremos cambiando los meses a su representación numérica. Esto lo hacemos para facilitar futuros análisis. Por otra parte, cambiaremos el nombre de algunos cultivos y de la columna *Entidad*, para homologar los nombres con los otros dataframes y facilitar su combinación.

In [39]:
month_to_number = {'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
                   'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 
                   'Diciembre': 12
                  }
df_temp['Mes'] = df_temp['Mes'].map(month_to_number)

change_crop_name = {"Tomate rojo (jitomate)": "Tomate rojo", "Toronja (pomelo)":"Toronja", 
                    "Nopalitos": "Nopal"}
df_temp['Cultivo'] = df_temp['Cultivo'].map(lambda x: change_crop_name.get(x, x))

change_state_name = {"México": "Estado de México"}
df_temp['Entidad'] = df_temp['Entidad'].map(lambda x: change_state_name.get(x, x))

df_temp = df_temp.rename(columns={'Entidad': 'Estado'})

In [40]:
df_temp.head()

,Año,Mes,Ciclo,Modalidad,Cultivo,Estado,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)
0,2020,1,Ciclicos - Perennes,Riego + Temporal,Tomate rojo,Aguascalientes,Aguascalientes,Calvillo,16.00,6.00,0.0,90.00,15.00
1,2020,1,Ciclicos - Perennes,Riego + Temporal,Tomate rojo,Baja California,Ensenada,Ensenada,19.50,0.00,0.0,0.00,0.00
2,2020,1,Ciclicos - Perennes,Riego + Temporal,Tomate rojo,Baja California Sur,Mulegé,Mulegé,80.00,0.00,0.0,0.00,0.00
3,2020,1,Ciclicos - Perennes,Riego + Temporal,Tomate rojo,Baja California Sur,Comondú,Comondú,127.00,0.00,0.0,0.00,0.00
4,2020,1,Ciclicos - Perennes,Riego + Temporal,Tomate rojo,Baja California Sur,La Paz,La Paz,611.00,106.00,0.0,"4,429.76",41.79


Ahora veremos los datos generales del dataframe.

In [41]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355334 entries, 0 to 355333
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Año                         355334 non-null  int64  
 1   Mes                         355334 non-null  int64  
 2   Ciclo                       355334 non-null  object 
 3   Modalidad                   355334 non-null  object 
 4   Cultivo                     355334 non-null  object 
 5   Estado                      355334 non-null  object 
 6   Distrito                    355334 non-null  object 
 7   Municipio                   355334 non-null  object 
 8   Superficie(ha)_Sembrada     355334 non-null  object 
 9   Superficie(ha)_Cosechada    355334 non-null  object 
 10  Superficie(ha)_Siniestrada  355334 non-null  object 
 11  Producción                  355334 non-null  object 
 12  Rendimiento(udm/ha)         355334 non-null  float64
dtypes: float64(1),

Observamos que no tenemos ningún dato faltante. Por otro lado, la columna *Superficie(ha)_Sembrada* es de tipo 'object'; sin embargo, las columnas *Superficie(ha)_Sembrada*, *Superficie(ha)_Cosechada*, *Superficie(ha)_Siniestrada*, *Producción* y *Rendimiento(udm/ha)* deben ser de tipo numérico. La razón de esta situación es la separación por comas de los números, como se muestra en la siguiente línea:

In [42]:
df_temp[(df_temp["Año"] == 2022) & (df_temp["Mes"] == 5) 
        & (df_temp["Municipio"] == "Villa de Cos") & (df_temp["Cultivo"] == "Chile verde")].head()

,Año,Mes,Ciclo,Modalidad,Cultivo,Estado,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)
212037,2022,5,Ciclicos - Perennes,Riego + Temporal,Chile verde,Zacatecas,Zacatecas,Villa de Cos,"2,619.35",0.00,0.0,0.00,0.0


Simplemente quitamos las comas de todos los números de estas columnas.

In [43]:
numeric_columns = ["Superficie(ha)_Sembrada", "Superficie(ha)_Cosechada", "Superficie(ha)_Siniestrada",
                   "Producción", "Rendimiento(udm/ha)"]
df_temp[numeric_columns] = df_temp[numeric_columns].applymap(custom_to_float)

In [44]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355334 entries, 0 to 355333
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Año                         355334 non-null  int64  
 1   Mes                         355334 non-null  int64  
 2   Ciclo                       355334 non-null  object 
 3   Modalidad                   355334 non-null  object 
 4   Cultivo                     355334 non-null  object 
 5   Estado                      355334 non-null  object 
 6   Distrito                    355334 non-null  object 
 7   Municipio                   355334 non-null  object 
 8   Superficie(ha)_Sembrada     355334 non-null  float64
 9   Superficie(ha)_Cosechada    355334 non-null  float64
 10  Superficie(ha)_Siniestrada  355334 non-null  float64
 11  Producción                  355334 non-null  float64
 12  Rendimiento(udm/ha)         355334 non-null  float64
dtypes: float64(5),

In [45]:
df_temp[(df_temp["Año"] == 2022) & (df_temp["Mes"] == 5) 
        & (df_temp["Municipio"] == "Villa de Cos") & (df_temp["Cultivo"] == "Chile verde")].head()

,Año,Mes,Ciclo,Modalidad,Cultivo,Estado,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)
212037,2022,5,Ciclicos - Perennes,Riego + Temporal,Chile verde,Zacatecas,Zacatecas,Villa de Cos,2619.35,0.0,0.0,0.0,0.0


Tambien eliminaremos las columnas 'Ciclo' y 'Modalidad', ya que el cliente solo le interesa el 'ciclo' *Ciclicos - Perennes* y la 'Modalidad' *Riego + Temporal*.

In [46]:
df_temp = df_temp.drop(['Ciclo', 'Modalidad'], axis=1)

In [47]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355334 entries, 0 to 355333
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Año                         355334 non-null  int64  
 1   Mes                         355334 non-null  int64  
 2   Cultivo                     355334 non-null  object 
 3   Estado                      355334 non-null  object 
 4   Distrito                    355334 non-null  object 
 5   Municipio                   355334 non-null  object 
 6   Superficie(ha)_Sembrada     355334 non-null  float64
 7   Superficie(ha)_Cosechada    355334 non-null  float64
 8   Superficie(ha)_Siniestrada  355334 non-null  float64
 9   Producción                  355334 non-null  float64
 10  Rendimiento(udm/ha)         355334 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 29.8+ MB


Por último, vemos que no tenemos instancias repetidas.

In [48]:
df_temp[df_temp.duplicated()]

,Año,Mes,Cultivo,Estado,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)


Ahora que estos datos están de la forma que queremos, podemos agregarlos a uno data frame general con todos los datos previamente limpiados.

In [49]:
file_SIAP = "SIAP.csv"
update_df_SIAP(file_SIAP, df_temp, replace=True)